In [10]:

import nltk
import requests
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kjjkjj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# nltk_utils.py
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import json
import numpy as np

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out


In [13]:
import torch.nn.functional as F
class ChatDataset(Dataset):
    def __init__(self,intents_file):
        with open(intents_file, 'r') as f:
            intents = json.load(f)
        self.intents = intents['intents']
        self.all_words = []
        self.tags = []
        self.xy = []
        for intent in self.intents:
            tag = intent['tag']
            self.tags.append(tag)
            for pattern in intent['patterns']:
                w = tokenize(pattern)
                self.all_words.extend(w)
                self.xy.append((w, tag))
        self.ignore_words = ['?', '.', '!']
        self.all_words = [stem(w) for w in self.all_words if w not in self.ignore_words]
        self.all_words = sorted(set(self.all_words))
        self.tags = sorted(set(self.tags))
        self.n_classes = len(self.tags)
        
        self.x_data = []
        self.y_data = []
        for (pattern_sentence, tag) in self.xy:
            bag = bag_of_words(pattern_sentence, self.all_words)
            label = self.tags.index(tag)
            self.x_data.append(bag)
            self.y_data.append(label)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]



In [14]:


# specify device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# create training dataset
train_dataset = ChatDataset('intents.json')

# define hyperparameters
input_size = len(train_dataset.all_words)
hidden_size = 40
output_size = train_dataset.n_classes
learning_rate = 0.0001
batch_size = 40
num_epochs = 500

# create dataloader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
# create neural network
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train the model

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch+1) % 10 == 0:
    
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.10f}')


print('Finished training')

# save the trained model
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": train_dataset.all_words,
"tags": train_dataset.tags
}

FILE = "./model.pth"
torch.save(data, FILE)


Epoch [10/500], Loss: 2.6477864735
Epoch [20/500], Loss: 1.7428750974
Epoch [30/500], Loss: 1.1636036034
Epoch [40/500], Loss: 0.8151883941
Epoch [50/500], Loss: 0.6143500478
Epoch [60/500], Loss: 0.4778442298
Epoch [70/500], Loss: 0.3860469501
Epoch [80/500], Loss: 0.3168461173
Epoch [90/500], Loss: 0.2636194138
Epoch [100/500], Loss: 0.2361908162
Epoch [110/500], Loss: 0.2051585512
Epoch [120/500], Loss: 0.1770047836
Epoch [130/500], Loss: 0.1579088687
Epoch [140/500], Loss: 0.1441864513
Epoch [150/500], Loss: 0.1343707059
Epoch [160/500], Loss: 0.1161682269
Epoch [170/500], Loss: 0.1111692761
Epoch [180/500], Loss: 0.1025108124
Epoch [190/500], Loss: 0.0958330432
Epoch [200/500], Loss: 0.0941705985
Epoch [210/500], Loss: 0.0835107295
Epoch [220/500], Loss: 0.0780736200
Epoch [230/500], Loss: 0.0743913496
Epoch [240/500], Loss: 0.0715982203
Epoch [250/500], Loss: 0.0670579222
Epoch [260/500], Loss: 0.0685842690
Epoch [270/500], Loss: 0.0628020431
Epoch [280/500], Loss: 0.0634084471
E

In [7]:
# chat.py
import random
import json
import requests
import torch

isFile = False
FILE = "./model.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

ignore_tag= ['greeting','goodbye','thanks','noanswer','options','Identity','Request']


with open('intents.json', 'r') as json_data:
  intents = json.load(json_data)

with open('suggestion.json', 'r') as suggestion:
    suggestionData = json.load(suggestion)

bot_name = "Health Bot"
print("Hi, can you describe where you feel discomfort or pain? (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit" or sentence == 'q':
        break
    print(sentence)
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    # print(prob)
    if prob.item() > 0.80:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
                if(tag not in ignore_tag):
                    print(f"here is some suggest : {suggestionData[tag]}")
    elif prob.item() > 0.65:
        if(tag not in ignore_tag):
            for intent in intents['intents']:
                if tag == intent["tag"]:
                    print(f"{bot_name}: I don't quite understand your question, but it may be a {random.choice(intent['responses'])}")
                    
        else:
            print(f"{bot_name}: I do not understand...")
    else:
        print(f"{bot_name}: I do not understand...")

Hi, can you describe where you feel discomfort or pain? (type 'quit' to exit)
I felt something wrong with my head.
Health Bot: head ache
here is some suggest : {'Description': 'A headache is a pain or discomfort that occurs in the head, scalp, or neck region. It can be classified into several types, including tension headaches, migraines, cluster headaches, and sinus headaches. Headaches can be caused by various factors, such as stress, anxiety, dehydration, lack of sleep, poor posture, eyestrain, and certain medical conditions.', 'SuggestionsText': 'Here are some suggestions for managing a headache', 'Suggestions': ['Drink plenty of water: Dehydration is a common cause of headaches, so make sure you drink enough water throughout the day.', 'Take a break: If you have been sitting at a desk or staring at a screen for an extended period, take a break and do some stretches.', 'Apply heat or cold: Applying heat or cold to the affected area can help relieve pain. Try using a hot compress or